In [1]:

from dotenv import load_dotenv
import sys
import os


load_dotenv()
fin_app_path = os.path.abspath('..')
sys.path.append(fin_app_path)

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.pydantic import PydanticOutputParser

from langchain.prompts import ChatPromptTemplate 
from dateutil.relativedelta import relativedelta

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
from datetime import datetime
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from langchain.agents.agent_types import AgentType


from pydantic import BaseModel, Field
from typing import Literal, List
import yfinance as yf
from IPython.display import Markdown, display


from langchain.tools import tool

from app.consts import l_llm, create_prompt, run_obj_to_dict

from app.chains import get_primary_chain
from app.consts import tags_struct_info, tags_struct_fin_annual, tags_struct_fin_quarterly, tags_struct_market

In [3]:
l = tags_struct_info + tags_struct_fin_annual + tags_struct_fin_quarterly + tags_struct_market

class temp(BaseModel):
    """ 
    Classify the input into one or more of the categories
    """ 
    r    : List[Literal[*l]]



In [42]:

p_chain = get_primary_chain(LLM=l_llm["o1-mini"])

query = "How has the Debt to Equity and Price to Earnings ratio of Varun Beverages changed over the last 3 quarters. How has this trend been across its industry"
res = p_chain.invoke(query)
print(res)




{'company': 'Varun Beverages', 'ticker': 'VBL.NS', 'required_data': ['quarterly_balance_sheet', 'quarterly_income_stmt', 'history'], 'RAG_tags': ['Financial Metrics', 'Sectoral Phenomenon'], 'start_date': '2024-04-01', 'end_date': '2024-12-31'}


In [29]:


def get_information(state):
    req_data = state['required_data']
    
    if len(req_data)==0:
        return {}
    else:
        stock = yf.Ticker(state['ticker'])

    intersection = lambda a,b : list(set(a) & set(b))


    l_fields_req_info = intersection(req_data, tags_struct_info)
    l_fields_req_fin = intersection(req_data, tags_struct_fin_annual+tags_struct_fin_quarterly)
    l_fields_req_market = intersection(req_data, tags_struct_market)

    d_info = {}
    l_df = []


    if len(l_fields_req_info)>0:
        for field in l_fields_req_info:
            if field in stock.info.keys():
                d_info[field] = stock.info[field]
    
    if len(l_fields_req_fin) > 0:
        for field in l_fields_req_fin:
            l_df.append(getattr(stock,field))
    
    if len(l_fields_req_market)>0:
        start_date = datetime.strptime(state['start_date'],'%Y-%M-%d')
        end_date = datetime.strptime(state['end_date'],'%Y-%M-%d')
        total_interval = (start_date - end_date).days
        if total_interval < 95: # Upto a little over 1 quarter
            hist_interval = '1d' 
        if total_interval < 400: # Upto a little over 1 year
            hist_interval = '1wk' 
        else:  # Over an year
            hist_interval = '1mo'

        hist = stock.history(start=start_date, end = end_date, interval=hist_interval)
        hist['percentage_change'] = hist.pct_change()['Close']

        l_df.append(hist)
    return {"list_info": d_info, "list_df": l_df}



In [69]:
state = {'company': 'Varun Beverages', 'ticker': 'VBL.NS', 
         'required_data': ['quarterly_balance_sheet','quarterly_balance_sheet','history'],
         'RAG_tags': ['Financial Metrics', 'Sectoral Phenomenon'], 
         'start_date': '2024-10-01', 'end_date': '2024-12-31'}



res =  get_information(state)
print(res)


{'list_info': {}, 'list_df': [                                                     2024-12-31  \
Treasury Shares Number                                      NaN   
Ordinary Shares Number                             3381510017.0   
Share Issued                                       3381510017.0   
Net Debt                                            979820000.0   
Total Debt                                        28262540000.0   
...                                                         ...   
Cash Cash Equivalents And Short Term Investments  32856700000.0   
Other Short Term Investments                      10193870000.0   
Cash And Cash Equivalents                         22662830000.0   
Cash Equivalents                                            NaN   
Cash Financial                                              NaN   

                                                     2024-06-30  \
Treasury Shares Number                                      0.0   
Ordinary Shares Number         

In [31]:
res['list_df']

[                                                        2024-12-31  \
 Tax Effect Of Unusual Items                                    0.0   
 Tax Rate For Calcs                                        0.232678   
 Normalized EBITDA                                    48308610000.0   
 Total Unusual Items                                            NaN   
 Total Unusual Items Excluding Goodwill                         NaN   
 Net Income From Continuing Operation Net Minori...   25946330000.0   
 Reconciled Depreciation                               9473860000.0   
 Reconciled Cost Of Revenue                           89047240000.0   
 EBITDA                                               48308610000.0   
 EBIT                                                 38834750000.0   
 Net Interest Income                                  -4503860000.0   
 Interest Expense                                      4503860000.0   
 Interest Income                                                NaN   
 Norma

In [49]:
query

'How has the Debt to Equity and Price to Earnings ratio of Varun Beverages changed over the last 3 quarters. How has this trend been across its industry'

In [80]:
def get_df_analysis_chain(LLM, l_df):

    df_analysis_system_prompt = """ 
    You are a data analysis bot helping a equity research analysts.
    You have been given a user query, and some data about the company's financial performance, and it's share market performane.
    Your primary objective is to provide the equity research analysts with  as many insights about the user-query and any relevant related analysis as possible.
    Do not invent any other information, not provided in the context.
    
    The query may request some ratios, or your analysis may be improved by using some commonly used financial ratios or metrics like Price to Earnings, some margings,  Debt to Equity, ROE, ROCE, EV/EBITDA -  In these cases, identify which fields would be required from which dataframe, join these dataframes if required, and perform the neccessary computations.

    Your response should be a markdown formatted string.
    Provide percentages where applicable. 
    For the stock price or relevant metrics, identify any major positive or negative %age changes, and thir time persiods, and add them to your analysis if relevant to user query. 
    Use you judgement on which metrics and their change and on which period may be most relevant to user query.     

    For currency numbers, use format ₹_ M, B, K for million, billion or thousand in your response.
    Wherever you think a table may be able to showcase your analysis better, please add markdown formatted tables in your response.
    Be structured in your analysis, and help the user understand the business impact of the numbers.

    MOST IMPORTANT INSTRUCTION: Only show analysis that can be directly computed from the data presented to you. Do not, under any circumstance, use any other information outside the data presented to you.   

    """
    agent = create_pandas_dataframe_agent(
        LLM,
        l_df,
        verbose=False,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        allow_dangerous_code=True
    )

    model_name = LLM.model_name
    system_profile = 'human' if  'o1-mini' in model_name  else 'system' 
    prompt = ChatPromptTemplate.from_messages(
    [
        (system_profile, df_analysis_system_prompt),
        ("user", '{input}'),
        ("placeholder", "{agent_scratchpad}"),
    ]
    )

    chain  = prompt | agent 
    return chain



In [ ]:
# llm_o1 = ChatOpenAI(model="o1-mini")
# LLM = llm_o1 
# model_name = LLM.model_name
# system_profile = 'human' if  'o1-mini' in model_name  else 'system' 
# system_profile

'human'

In [79]:
llm_o1 = ChatOpenAI(model="o1-mini")

chain = get_df_analysis_chain(LLM = llm_o1, l_df= res["list_df"])
r1 = chain.invoke(query)

BadRequestError: Error code: 400 - {'error': {'message': "Unsupported value: 'messages[0].role' does not support 'system' with this model.", 'type': 'invalid_request_error', 'param': 'messages[0].role', 'code': 'unsupported_value'}}

In [71]:
llm_4o = ChatOpenAI(model="gpt-4o")
chain = get_df_analysis_chain(LLM = llm_4o, l_df= res["list_df"])

r2 = chain.invoke(query)

In [72]:
display(Markdown(r2['output']))




### Stock Price Movement Analysis

Examining the stock price movements provides insights into how the market perceives the company's performance and risk:

| Date       | Percentage Change |
|------------|-------------------|
| 2024-01-08 | -1.10%            |
| 2024-01-29 | +1.83%            |

#### Observations:
- **2024-01-08**: There was a notable decrease of 1.10% in the stock price, indicating potential negative sentiment or reaction to news regarding Varun Beverages.
- **2024-01-29**: The stock saw a significant increase of 1.83%, which may indicate positive market sentiment or favorable news affecting Varun Beverages.

### Conclusions:

- The high Debt to Equity ratio of 20.81 in 2024-06-30 suggests that Varun Beverages relied heavily on debt for its financing needs during this period.
- In terms of market performance, notable fluctuations in stock price were observed in January 2024, which could be indicative of varying investor sentiments or external factors affecting the company.
- Since the P/E ratio could not be computed due to missing Net Income data, further analysis on company profitability could not be conducted with the provided data.

For a thorough industry trend analysis, comparative data from similar companies in the same industry would be needed, which is not available in the current dataset.

In [89]:
s = res['list_df'][0].to_markdown()
print(s)

|                                                                      |   2024-12-31 00:00:00 |   2024-06-30 00:00:00 |   2023-12-31 00:00:00 |   2023-06-30 00:00:00 |
|:---------------------------------------------------------------------|----------------------:|----------------------:|----------------------:|----------------------:|
| Treasury Shares Number                                               |         nan           |           0           |         nan           |         nan           |
| Ordinary Shares Number                                               |           3.38151e+09 |           3.24862e+09 |           3.24804e+09 |           3.24784e+09 |
| Share Issued                                                         |           3.38151e+09 |           3.24862e+09 |           3.24804e+09 |           3.24784e+09 |
| Net Debt                                                             |           9.7982e+08  |           6.07639e+10 |           4.95218e+10 |           

In [ ]:
def get_generation_chain(LLM):
    prompt =ChatPromptTemplate.from_messages([
        ("human","""
         You are an equity research analyst tasked with answering user queries. Your primary goal is to answer the user query.
         To help you, you've been provided several analysis from your assistants. 
         This may include one or more of the following in your context
         - Extra Information : fetched from company's ticker, 
         - Data Frames : financial statements and / or stock history of the company 
         - DF Analysis: An analyst interpretation of the financial or market data
         - RAG Passages: relevant from the company filings (annual reports, transcripts, PPTs etc)
         
         You are required to synthesise all of this information to answer the user's original query.
         Do not invent any data apart from the above data provided to you in the context
         Be structured and methadological in your response.
         Your answer should be markdown friendly 
        """),
        ("ai","Extra Information : {list_info}"),
        ("ai","Data Frames : {list_df}"),
        ("ai","DF Analysis : {df_analysis}"),
        ("ai","RAG Passages : {rag_data}"),
        ("human","Query : {user_query}"),
    ])
    chain =  prompt | LLM | StrOutputParser()
    return chain


In [68]:
state

{'company': 'Varun Beverages',
 'ticker': 'VBL.NS',
 'required_data': ['income_stmt', 'website', 'history'],
 'RAG_tags': ['Financial Metrics', 'Sectoral Phenomenon'],
 'start_date': '2024-10-01',
 'end_date': '2024-12-31'}

In [ ]:

# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="o1-mini")
# model.invoke("Count the number of letter 'r' in the word 'strawberry' ")

AIMessage(content='The word **"strawberry"** contains **three** letter \'r\'s. Here\'s the breakdown:\n\n- **s**\n- **t**\n- **r** ← 1st \'r\'\n- **a**\n- **w**\n- **b**\n- **e**\n- **r** ← 2nd \'r\'\n- **r** ← 3rd \'r\'\n- **y**\n\nSo, there are **3** instances of the letter \'r\' in "strawberry."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 499, 'prompt_tokens': 24, 'total_tokens': 523, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_5c373483f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-3aecfcba-12a2-40f4-bbdf-1abb2ebcba7f-0', usage_metadata={'input_tokens': 24, 'output_tokens': 499, 'total_tokens': 523, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [9]:
res

primaryClf(company='Varun Beverages', ticker='VBL.NS', required_data=['income_stmt'], RAG_tags=['Financial Metrics'], start_date='2020-03-01', end_date='2025-03-03')

### Trying Lang Graph graphs 

In [62]:
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver

from langchain_core.runnables import RunnablePassthrough, RunnableLambda


In [ ]:
from typing import TypedDict, List , Annotated, Dict, Any
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, HumanMessage




In [54]:

# Defining state of Graph

class GraphState(TypedDict):
    """
    Represents the state of the  graph.

    Attributes:
        query: User Query
        company: Name of company in user query
        ticker: Ticker of comapny in user query
        information_type: Type of user query
        required_data: Information required from yfinance API
        time_duration: time_duration extracted from the user query
        generation: LLM generated Response
        messages: List of All Messages

    """

    query: str
    company: str
    ticker:str
    information_type:str
    required_data: List[str]
    time_duration: str
    generation: str
    messages:  Annotated[list, add_messages]


In [61]:
#### Chains
primary_chain = get_primary_chain()

##### Langgraph NODES

In [ ]:
def human_node(state: GraphState)-> Dict[str, Any]:
    return {'messages':HumanMessage(state['query'])} 



###  Testing YF Tool Agent

In [ ]:
import yfinance as yf

stock = yf.Ticker('ASIANPAINT.NS')




In [ ]:
# from datetime import datetime
# datetime.today().strftime('%Y-%m-%d')

'2025-02-19'

In [22]:
str_keys = ['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']

"""
    Fetches information about the ticker mentioned. 
    argument l_fields is the set of relevant fields to chosen from the following list - {%s}
    """%(str(str_keys))


"\n    Fetches information about the ticker mentioned. \n    argument l_fields is the set of relevant fields to chosen from the following list - {['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']}\n    "

In [28]:
str_keys = ['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']


@tool
def gen_info_tool(ticker, l_fields) -> str:
    """
    Fetches information about the ticker mentioned. 
    argument l_fields is the set of relevant fields to chosen from the following list - 
    ['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']    "
    """
    stock = yf.Ticker(ticker)
    d_ret = {}
    for field in l_fields:
        if field in stock.info.keys():
            d_ret[field] = stock.info[field]
    return str(d_ret)

    

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You have to give response to the users query, given the following information"),
        ("ai", "{context}"),
        ("human", "Query : {input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [gen_info_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

query = "What is the full address of Varun Beverages?"

agent_executor.invoke({"input": query, "context": res.json()})

{'input': 'What is the full address of Varun Beverages?',
 'context': '{"company": "Varun Beverages", "ticker": "VBL.NS", "information_type": "General", "time_duration": "NA"}',
 'output': 'The full address of Varun Beverages is:\n\nRJ Corp House,  \nPlot No. 31 Institutional Area Sector-44,  \nGurugram,  \n122002,  \nIndia.'}

## Testing Pandas DF Agent
 

In [16]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType



In [29]:
# df_stock = stock.history()
# df_stock = 
stock.income_stmt

,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Tax Effect Of Unusual Items,560574715.866174,285503565.759567,297421093.817161,308292401.872524,NaN
Tax Rate For Calcs,0.243622,0.262532,0.263368,0.254998,NaN
Normalized EBITDA,81758400000.0,65578900000.0,49730900000.0,50663500000.0,NaN
Total Unusual Items,2301000000.0,1334100000.0,1264000000.0,1209000000.0,NaN
Total Unusual Items Excluding Goodwill,2301000000.0,1087500000.0,1129300000.0,1209000000.0,NaN
Net Income From Continuing Operation Net Minority Interest,54602300000.0,41064500000.0,30305700000.0,31392900000.0,NaN
Reconciled Depreciation,8530000000.0,8580200000.0,8163600000.0,7912700000.0,NaN
Reconciled Cost Of Revenue,201743000000.0,212462900000.0,183775400000.0,121567400000.0,NaN
EBITDA,84059400000.0,66913000000.0,50994900000.0,51872500000.0,NaN
EBIT,75529400000.0,58332800000.0,42831300000.0,43959800000.0,NaN


In [19]:
agent = create_pandas_dataframe_agent(
    llm,
    df_stock,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
         You are a equity research analyst given information about the company's stock data. 
         Answer the user's query with the most relevant information a stock trader may be concerned about.
         Try keeping your answer as concise as possible.         
         """),
        ("human", "Query : {input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
res = agent.invoke("How has the share performed over the last 5 days?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# We will calculate the percentage change from the first day to the last of the 'Close' prices over the last 5 days to assess the performance.\n\n# Get the first and last close prices in the last 5 days\nfirst_close_price = df['Close'].iloc[0]\nlast_close_price = df['Close'].iloc[4]\n\n# Calculate the change\npercentage_change = ((last_close_price - first_close_price) / first_close_price) * 100\npercentage_change"}`


-0.795867053814564Over the last 5 days, the share's closing price has decreased by approximately 0.80%.

> Finished chain.


In [27]:
res

{'input': 'How has the share performed over the last 5 days?',
 'output': "Over the last 5 days, the share's closing price has decreased by approximately 0.80%."}

### Now testing for financial data


In [40]:


agent_fin = create_pandas_dataframe_agent(
    llm,
    stock.income_stmt,
    verbose=False,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True

)
prompt_fin = ChatPromptTemplate.from_messages(
    [
        ("system", """
         You are a equity research analyst given information about the company's Financial Statements. 
         Answer the user's query with the most relevant information an investor may be concerned about.
         Quote relevant ratios where required.
         Try keeping your answer concise.
         """),
        ("human", "Query : {input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


In [41]:
query = "How have earnings performed over the past 3 years?"

chain = prompt_fin | agent_fin 
res = chain.invoke(query)

print(res['output'])

Error in LangChainTracer.on_tool_end callback: TypeError('keys must be str, int, float, bool or None, not Timestamp')
Error in LangChainTracer.on_chain_end callback: TypeError('keys must be str, int, float, bool or None, not Timestamp')
Error in LangChainTracer.on_chain_start callback: TypeError('keys must be str, int, float, bool or None, not Timestamp')


Over the past three years, the company's earnings, as measured by Normalized EBITDA, have shown an upward trend:

- From 2022 to 2023, Normalized EBITDA increased from approximately $49.73 billion to $65.58 billion, representing a growth of around 31.8%.
- From 2023 to 2024, further growth occurred as Normalized EBITDA rose from approximately $65.58 billion to $81.76 billion, resulting in an increase of about 24.6%.

Overall, there has been consistent and substantial growth in earnings over the past three years, indicating strong performance.


In [35]:
print(res["output"])

Over the past three years, the company's earnings, measured by Normalized EBITDA, have shown a strong upward trend:

- FY 2022: $49.73 billion
- FY 2023: $65.58 billion
- FY 2024: $81.76 billion

This indicates robust growth in earnings, increasing by approximately 32% from FY 2022 to FY 2023 and about 25% from FY 2023 to FY 2024. This performance reflects positively on the company's operational profitability.
